In [1]:
from src.neural_networks.get_data import get_tf_data, stack_1d_features, normalize_1d_features_with_mean_std, load_cv_data, make_importance_sampling_dataset
from src.neural_networks.nn_forecast import NNForecast
from src.visualization.twcrpss_plot import make_twcrpss_plot_tf
from src.visualization.brier_score import make_brier_skill_plot_tf
from src.visualization.pit import make_cpit_diagram_tf, comp_multiple_pit_scores
from src.visualization.reliability_diagram import make_reliability_and_sharpness_tf
from src.training.training import load_model
from src.visualization.plot_forecasts import plot_forecast_pdf_tf
from src.climatology.climatology import Climatology
from src.visualization.brier_score import get_brier_scores_tf
from src.models.emos import BootstrapEmos, EMOS

import sklearn

from scipy.stats import norm, pareto

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pickle
import optuna

2024-06-21 10:05:56.871635: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-21 10:05:56.874361: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-21 10:05:56.908408: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-21 10:06:02.805300: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
all_features = ['wind_speed', 'press', 'kinetic', 'humid', 'geopot']

location_features = ['wind_speed', 'press', 'kinetic', 'humid', 'geopot']

scale_features = ['wind_speed', 'press', 'kinetic', 'humid', 'geopot']

features_names_dict = {name: 1 for name in all_features}

features_names_dict['wind_speed'] = 15

ignore = ['229', '285', '323']

In [ ]:
train_data, test_data, data_info = load_cv_data(3, features_names_dict)

train_data_is = make_importance_sampling_dataset(train_data)

train_data_is = train_data_is.shuffle(30000)

train_data_is = train_data_is.batch(32)

train_data_is = train_data_is.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
train_data = train_data.shuffle(30000)

train_data = train_data.batch(32)

train_data = train_data.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
loss = "loss_CRPS_sample"
#loss = "loss_cPIT"
samples = 200

# possible chain functions: 'chain_function_indicator' and 'chain_function_normal_cdf'
# if chain_function_indicator is chosen, threshold is not necessary
# if chain_function_normal_cdf is chosen, threshold is necessary
chain_function = "chain_function_normal_cdf_plus_constant"
threshold = 100
chain_function_mean = 9
chain_function_std = 0.5
chain_function_constant = 0.001


# possible optimizers: 'SGD', 'Adam'
optimizer = "Adam"
learning_rate = 0.01

# possible forecast distributions: 'distr_trunc_normal', 'distr_log_normal', 'distr_gev' and 'distr_mixture'/'distr_mixture_linear', which can be a mixture distribution of two previously mentioned distributions.
forecast_distribution = "distr_trunc_normal"

# necessary in case of a mixture distribution
distribution_1 = "distr_trunc_normal"
distribution_2 = "distr_log_normal"

random_init = False
printing = True
subset_size = None

setup = {'loss': loss,
         'samples': samples, 
         'optimizer': optimizer, 
         'learning_rate': learning_rate, 
         'forecast_distribution': forecast_distribution,
         'chain_function': chain_function,
         'threshold': threshold,
         'distribution_1': distribution_1,
         'distribution_2': distribution_2,
         'chain_function_mean': chain_function_mean,
         'chain_function_std': chain_function_std,
         'chain_function_constant': chain_function_constant,
         'all_features': all_features,
         'location_features': location_features,
         'scale_features': scale_features,
         'random_init': random_init,
         'subset_size': subset_size,
        'printing': printing,
         }

In [ ]:
emos_is = EMOS(setup)

emos_base = EMOS(setup)

In [ ]:
epochs = 30

losses_is = emos_is.fit(train_data_is, epochs, True)['time_hist']

losses = emos_base.fit(train_data, epochs, True)['time_hist']

In [ ]:
plt.plot(list(losses_is.keys()), list(losses_is.values()), label='Importance Sampling')
plt.plot(list(losses.keys()), list(losses.values()), label='Regular Sampling')
plt.xlabel('Seconds')
plt.ylabel("Loss Values")
plt.grid()
plt.legend()
plt.title("Models trained on the CRPS")
plt.show()

In [ ]:
setup['loss'] = 'loss_twCRPS_sample'

In [ ]:
emos_weight_is = EMOS(setup)

emos_weight_base = EMOS(setup)

In [ ]:
epochs = 30

losses_is_w = emos_weight_is.fit(train_data_is, epochs, True)['time_hist']

losses_w = emos_weight_base.fit(train_data, epochs, True)['time_hist']

In [ ]:
plt.plot(list(losses_is_w.keys()), list(losses_is_w.values()), label='Importance Sampling')
plt.plot(list(losses_w.keys()), list(losses_w.values()), label='Regular Sampling')
plt.xlabel('Seconds')
plt.ylabel("Loss Values")
plt.grid()
plt.legend()
plt.title("Models trained on the twCRPS")
plt.show()